### Deepfakes - Travaux Jupyter Notebook pour ADS

Ce projet à pour but de montrer deux moyens de créer des deepfakes. Leur usage est le même :

- Les Autoencodeurs
- Les Réseaux Génératifs Adversariaux (GAN)

Tous deux ont pour but de produire la même chose : une imitation de quelque chose selon des images données.

Dans les deux exemples, les modèles seront entraînés avec CelebA, une liste de têtes de célébrités connues, disponibles [ici](https://www.kaggle.com/jessicali9530/celeba-dataset)

Pour les explications sur le fonctionnement des modèles, référez-vous au papier de l'ADS.

### Partie 2 - GAN

In [1]:
# Imports
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import matplotlib.pyplot as plt
import os

output = "generated_images" # Folder for generated images over epochs

In [2]:
dataset_directory = "img_align_celeba" # input directory

In [11]:
btch_size = 32
img_size = 64
dataset = keras.preprocessing.image_dataset_from_directory(
    dataset_directory, label_mode=None, image_size=(img_size, img_size), batch_size = btch_size
)

dataset = dataset.map(lambda x: x / 255.0).shuffle(btch_size).repeat()

Found 202599 files belonging to 1 classes.


In [4]:
# The discriminator will be the one that'll judge if the image
# done by the generator is real or fake.
# It is made thanks to the LeakyReLU architecture.
discriminator = keras.Sequential(
    [
        keras.Input(shape=(64, 64, 3)),
        layers.Conv2D(64, kernel_size=4, strides=2, padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv2D(128, kernel_size=4, strides=2, padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv2D(128, kernel_size=4, strides=2, padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.Flatten(),
        layers.Dropout(0.2),
        layers.Dense(1, activation="sigmoid"),
    ],
    name="discriminator",
)
discriminator.summary()

Model: "discriminator"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 32, 32, 64)        3136      
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 32, 32, 64)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 16, 16, 128)       131200    
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 16, 16, 128)       0         
                                                                 
 conv2d_2 (Conv2D)           (None, 8, 8, 128)         262272    
                                                                 
 leaky_re_lu_2 (LeakyReLU)   (None, 8, 8, 128)         0         
                                                                 
 flatten (Flatten)           (None, 8192)            

In [5]:
# The generator is a fully-convolutional network
# that is going to generate the fake pictures
# using the Conv2DTranspose (that will take the seed from the layers)
# to generate the final picture.
# Decodes with the help of given layers, and outputs an upsampled image
latent_dim = 512

generator = keras.Sequential(
    [
        keras.Input(shape=(latent_dim,)),
        layers.Dense(8 * 8 * 128),
        layers.Reshape((8, 8, 128)),
        layers.Conv2DTranspose(128, kernel_size=4, strides=2, padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv2DTranspose(256, kernel_size=4, strides=2, padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv2DTranspose(512, kernel_size=4, strides=2, padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv2D(3, kernel_size=5, padding="same", activation="sigmoid"),
    ],
    name="generator",
)
generator.summary()

Model: "generator"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_1 (Dense)             (None, 8192)              4202496   
                                                                 
 reshape (Reshape)           (None, 8, 8, 128)         0         
                                                                 
 conv2d_transpose (Conv2DTra  (None, 16, 16, 128)      262272    
 nspose)                                                         
                                                                 
 leaky_re_lu_3 (LeakyReLU)   (None, 16, 16, 128)       0         
                                                                 
 conv2d_transpose_1 (Conv2DT  (None, 32, 32, 256)      524544    
 ranspose)                                                       
                                                                 
 leaky_re_lu_4 (LeakyReLU)   (None, 32, 32, 256)       0 

In [6]:
class GAN(keras.Model):
    def __init__(self, discriminator, generator, latent_dim):
        super(GAN, self).__init__()
        self.discriminator = discriminator
        self.generator = generator
        self.latent_dim = latent_dim

    def compile(self, d_optimizer, g_optimizer, loss_fn):
        super(GAN, self).compile()
        self.d_optimizer = d_optimizer
        self.g_optimizer = g_optimizer
        self.loss_fn = loss_fn
        self.d_loss_metric = keras.metrics.Mean(name="d_loss")
        self.g_loss_metric = keras.metrics.Mean(name="g_loss")

    @property
    def metrics(self):
        return [self.d_loss_metric, self.g_loss_metric]

    def train_step(self, real_images):
        # Nous prenons des images aléatoires dans notre espace latent
        batch_size = tf.shape(real_images)[0]
        random_latent_vectors = tf.random.normal(shape=(batch_size, self.latent_dim))

        # Nous les décodons en créant des fausses versions de ces images
        generated_images = self.generator(random_latent_vectors)

        # Nous les combinons avec des vraies images
        combined_images = tf.concat([generated_images, real_images], axis=0)

        # On assemble les labels, puis nous le présentons au discriminateur
        # qui devra déterminer le vrai du faux.
        labels = tf.concat(
            [tf.ones((batch_size, 1)), tf.zeros((batch_size, 1))], axis=0
        )
        # Ajout du bruit sur l'image (pour permettre de tromper le discriminateur !)
        labels += 0.05 * tf.random.uniform(tf.shape(labels))

        # On entraîne le discriminateur
        with tf.GradientTape() as tape:
            predictions = self.discriminator(combined_images)
            d_loss = self.loss_fn(labels, predictions)
        grads = tape.gradient(d_loss, self.discriminator.trainable_weights)
        self.d_optimizer.apply_gradients(
            zip(grads, self.discriminator.trainable_weights)
        )

        # Nous prenons des images aléatoires dans notre espace latent
        random_latent_vectors = tf.random.normal(shape=(batch_size, self.latent_dim))

        # Nous prenons tous les labels qui pensent croire que les images sont 'réelles'
        misleading_labels = tf.zeros((batch_size, 1))

        # On entraîne le générateur (sans mettre à jour les poids du discriminateur, c'est très important)
        with tf.GradientTape() as tape:
            predictions = self.discriminator(self.generator(random_latent_vectors))
            g_loss = self.loss_fn(misleading_labels, predictions)
        grads = tape.gradient(g_loss, self.generator.trainable_weights)
        self.g_optimizer.apply_gradients(zip(grads, self.generator.trainable_weights))

        # Mise à jour des métriques
        self.d_loss_metric.update_state(d_loss)
        self.g_loss_metric.update_state(g_loss)
        return {
            "d_loss": self.d_loss_metric.result(),
            "g_loss": self.g_loss_metric.result(),
        }

In [7]:
class GANMonitor(keras.callbacks.Callback):
    def __init__(self, num_img=3, latent_dim=128):
        self.num_img = num_img
        self.latent_dim = latent_dim

    def on_epoch_end(self, epoch, logs=None):
        random_latent_vectors = tf.random.normal(shape=(self.num_img, self.latent_dim))
        generated_images = self.model.generator(random_latent_vectors)
        generated_images *= 255
        generated_images.numpy()
        for i in range(self.num_img):
            img = keras.preprocessing.image.array_to_img(generated_images[i])
            img.save(f"{output}/generated_img_%03d_%d.png" % (epoch, i))

In [13]:
epochs = 100

gan = GAN(discriminator=discriminator, generator=generator, latent_dim=latent_dim)
gan.compile(
    d_optimizer=keras.optimizers.Adam(0.00007, 0.5),
    g_optimizer=keras.optimizers.Adam(0.00007, 0.5),
    loss_fn=keras.losses.BinaryCrossentropy(),
)

gan.fit(
    dataset, epochs=epochs, callbacks=[GANMonitor(num_img=10, latent_dim=latent_dim)], steps_per_epoch = 200
)

Epoch 1/100
200/200 [==============================] - 55s 261ms/step - d_loss: 0.6766 - g_loss: 0.7962
Epoch 2/100
200/200 [==============================] - 57s 286ms/step - d_loss: 0.6788 - g_loss: 0.7874
Epoch 3/100
200/200 [==============================] - 61s 308ms/step - d_loss: 0.6789 - g_loss: 0.7899
Epoch 4/100
200/200 [==============================] - 65s 325ms/step - d_loss: 0.6773 - g_loss: 0.7957
Epoch 5/100
200/200 [==============================] - 67s 337ms/step - d_loss: 0.6768 - g_loss: 0.7918
Epoch 6/100
200/200 [==============================] - 69s 345ms/step - d_loss: 0.6795 - g_loss: 0.7916
Epoch 7/100
200/200 [==============================] - 71s 355ms/step - d_loss: 0.6791 - g_loss: 0.7964
Epoch 8/100
200/200 [==============================] - 72s 359ms/step - d_loss: 0.6783 - g_loss: 0.7915
Epoch 9/100
200/200 [==============================] - 73s 366ms/step - d_loss: 0.6782 - g_loss: 0.7967
Epoch 10/100
200/200 [==============================] - 74s 371m

In [14]:
# Saving the model
generator.save('gen.h5')
discriminator.save('disc.h5')